# Workload generator

includes :

In [1]:
import json
import random
import gzip
from datetime import datetime
from math import floor

In [2]:
tracesPath='./clusterdata-2011-2'
tasksNums = range(0,100)

tasksFiles=list(map(lambda x:tracesPath+'/task_events/part-'+str(x).zfill(5)+'-of-00500.csv.gz', tasksNums))

print (tasksFiles[0])
print ('to')
print (tasksFiles[-1])

./clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz
to
./clusterdata-2011-2/task_events/part-00099-of-00500.csv.gz


In [3]:
def genJobs(taskfiles):
    tasks={}
    arrival=[]
    current_time=0
    jobs = []
    profiles = {}
    
    def task_complete(jID, tID):
        (res, prio, evts) = tasks[(jID, tID)]
        if len(evts) < 3:
            #bad_tasks.append((jID, tID))
            pass
        else:
            subtime=evts[0][1] #timestamp du debut
            runtime=evts[1][1] #event du lancement de la tache
            endtime=evts[-1][1] #fin de la tache
            
            if evts[-1][0]==4 and res!=0.0:        
                #temps en seconde
                subt=subtime/1000000.0
                #temps maximun
                walltime=float(endtime-runtime)/1000000.0

                #creation du job
                job={}
                job['id'] = str(jID) + "-" + str(tID) + "-" + str(subt)
                job['profile'] = str(floor(walltime))
                job['res'] = int(res*128)
                if int(res*128)==0:
                    job['res'] = 1 
                job['subtime'] = floor(subt)
                job['walltime'] = floor(walltime*2)

                jobs.append(job)
                
                #creation d'un profile avec le temps de la task
                profiles[str(floor(walltime))]={}
                profiles[str(floor(walltime))]['type']="delay"
                profiles[str(floor(walltime))]['delay']=floor(walltime)

                #print(json.dumps(profiles))
                #print(json.dumps(job))

                del tasks[(jID, tID)]
    
    for f in tasksFiles:
        print('File: ', f)
        with gzip.open(f, 'rb') as f:
            for line in f.readlines():
                
                """
                1. timestamp
                2. missing info
                3. job ID
                4. task index - within the job
                5. machine ID
                6. event type
                7. user name
                8. scheduling class
                9. priority
                10. resource request for CPU cores
                11. resource request for RAM
                12. resource request for local disk space
                13. different-machine constraint
                """
                
                (timestamp,_,jID,tID,_,evtType,_,_,prio,res,_,_,_)=line.decode("ascii").split(',')
                try:
                    # début de la tache
                    if(evtType == '0'):
                        timestamp=int(timestamp)
                        #on récupère la class, la prio, et le temps depuis le début, qu'on stock
                        tasks[(int(jID), int(tID))] = (float(res), int(prio), [(0,int(timestamp))])
                        #on actualise le temps
                        if timestamp>0:
                            arrival.append(timestamp-current_time)
                            current_time=timestamp
                            
                    # event de la tache en cours        
                    else:
                        
                        timestamp, jID, tID, evtType = int(timestamp), int(jID), int(tID), int(evtType)
                        (_,_, events)=tasks[(jID,tID)]
                        #on rajoute le timestamp de l'evenement
                        events.append((evtType,timestamp))
                        
                        #fin de la tache
                        if evtType == 4 or evtType == 5:
                            task_complete(jID, tID)
                            
                except:
                    pass
    
                
    return (jobs,profiles)

In [4]:
#j : jobs
#p : profiles
(j,p)= genJobs(tasksFiles)

File:  ./clusterdata-2011-2/task_events/part-00000-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00001-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00002-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00003-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00004-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00005-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00006-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00007-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00008-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00009-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00010-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00011-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00012-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00013-of-00500.csv.gz
File:  ./clusterdata-2011-2/task_events/part-00014-of-00500.cs

In [5]:
jsonPath = "./workloads/workload.json"

outputData = {}
outputData['description'] ="Workload from google data"
outputData['date'] = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
outputData['nb_res'] = 128

outputData['profiles'] = p
        

outputData['jobs'] = j[0:len(j):1000]

with open(jsonPath, "w") as jsonF:
    jsonF.write(json.dumps(outputData, indent = 4))